In [2]:
import os
os.chdir("./data")

In [3]:
import h5py
filename = '1034 SI_B, Aug 15, 13 7 49DeepCut_resnet50_ReachingMar12shuffle1_800.h5'
f = h5py.File(filename, 'r')

# List all groups
print("Keys: %s" % f.keys())
a_group_key = list(f.keys())[0]

# Get the data
data = list(f[a_group_key])

Keys: KeysView(<HDF5 file "1034 SI_B, Aug 15, 13 7 49DeepCut_resnet50_ReachingMar12shuffle1_800.h5" (mode r)>)


In [4]:
import pandas as pd
 
with pd.HDFStore('1034 SI_B, Aug 15, 13 7 49DeepCut_resnet50_ReachingMar12shuffle1_800.h5', 'r') as d:
    df = d.get(list(f.keys())[0])
    df.to_csv('1030 SI_B.csv')

In [5]:
data = pd.read_csv("1030 SI_B.csv",skiprows = 2)

#### Remove the first ten seconds, which has no mouse in the case (200 frames)

In [6]:
move_data = data.loc[200:]

In [7]:
#Distance bewteen defeated mouse head and encloser
import numpy as np 
def cal_dist(data): 
    fixedX = 50
    fixedY = 200
    x = data["x"]
    y = data["y"]
    dist = np.sqrt((x - fixedX)**2 + (y - fixedY)**2)  
    return dist  
cal_dist(move_data)  

200     352.954902
201     339.529008
202     330.710550
203     316.288294
204     308.267518
205     297.640930
206     294.311149
207     294.930426
208     303.958526
209     311.620245
210     323.272076
211     327.131556
212     330.177052
213     332.721866
214     337.193786
215     401.653015
216     389.209916
217     342.116334
218     339.216278
219     341.110168
220     347.052314
221     348.749794
222     346.468282
223     339.524622
224     334.626589
225     331.112141
226     329.172125
227     328.365842
228     329.069008
229     326.019257
           ...    
3451    127.740526
3452    134.342602
3453    143.512255
3454    150.176937
3455    155.363634
3456    158.923237
3457    163.255360
3458    167.230809
3459    175.416077
3460    186.652375
3461    186.788972
3462    182.617940
3463    182.016852
3464    186.696386
3465    193.843394
3466    195.998862
3467    202.692209
3468    207.597093
3469    218.553845
3470    224.174640
3471    226.791130
3472    231.

In [65]:
#Head Direction
# Reference
# https://stackoverflow.com/questions/42258637/how-to-know-the-angle-between-two-point
from math import degrees, atan2
def head_dir(data):
    p1 = pd.concat([data["x"], data["y"]],axis=1)
    p2 = pd.concat([data["x.1"], data["y.1"]], axis = 1, keys=['x', 'y'])
    xDiff = p2.x - p1.x
    yDiff = p2.y - p1.y
    degree = []
    for i in range(0,len(xDiff)):
        degree.append(degrees(atan2(tuple(yDiff)[i], tuple(xDiff)[i])))
    direction = [x-x+1 if x >= 90 or x <= -90 else x-x for x in degree]
    return (direction, degree)


[x[0:5] for x in head_dir(move_data)]
# Whether the mouse's direction towards bullying mouse or not, 1: yes; 0: no
# The degree of head direction, clockwise.

[[1.0, 1.0, 1.0, 1.0, 1.0],
 [105.06555330944097,
  103.2993416748217,
  119.87163049942879,
  114.83303281636094,
  115.31710226905174]]

In [67]:
#Anotations of bahavior definition

# Approaching the interaction zone (area around the enclosure)
# Avoiding the interaction zone - this could include instances where the animal enters the 1/2 of the chamber containing the enclosure + the head direction is facing the enclosure, but it doesn't enter the interaction zone
# Sniffing/directly interacting with the enclosure
# Sitting in the corners (2 corners closest to enclosure vs 2 far corners)

def annotate(interactionX = 125, interactionYUpper = 250, interactionYLower = 150, corUppL = 50, corUppR = 400, MidLineX = 235, enclosureX = 90, enclosureYUpper = 300, enclosureYLower = 200, data = data):
    posX = data['x'].tolist()
    posY = data['y'].tolist()
    head = head_dir(data)[0] #Whether the head is facing enclosure
    label = []
    for i in range(0, len(posX)):
        if posX[i] < interactionX and posY[i] < interactionYUpper and posY[i] > interactionYLower and head[i] == 1.0: #Approaching
            label.append(1)
        elif posX[i] <= MidLineX and posX[i] > interactionX and (posY[i] > interactionYUpper or posY[i] < interactionYLower) and head[i] == 1.0: #Avoidance
            label.append(2)
        elif posX[i] < enclosureX and posY[i] > enclosureYLower and posY[i] < enclosureYUpper: #Sniffing
            label.append(3)
        elif (posX[i] < corUppL and posY[i] < corUppL) or (posX[i] > corUppR and posY[i] < corUppL) or (posX[i] < corUppL and posY[i] > corUppR) or (posX[i] > corUppR and posY[i] > corUppR): #Sitting in the corners
            label.append(4)
        else: #Other cases
            label.append(5)
    return label
    

annotate(data = move_data)[0:5]

[5, 5, 5, 5, 5]